# TAHLR Week 14: Frequency Analysis

Code notebook for TAHLR course at ISAW (Fall 2023) based on Sweigart 2018 (CCWP) Ch. 19: Frequency Analysis and Ch. 6: Hacking the Caesar Cipher with Brute-Force

In [ ]:
# Imports

from collections import Counter
import re
import matplotlib.pyplot as plt
import urllib.request
from pprint import pprint

from detectEnglish import isEnglish

## Analyzing the Frequency of Letters in Text

In [ ]:
# Helper function, preprocesses text

def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-z]', '', text)
    return text

In [ ]:
# Get text of Moby-Dick

url = 'https://www.gutenberg.org/cache/epub/2701/pg2701.txt'
response = urllib.request.urlopen(url)
data = response.read().decode('utf-8')
text = preprocess(data.lower())
sample = [w.strip().lower() for w in text if w.strip()]
print(sample[993:1007])

In [ ]:
# Get counter

letter_frequency = Counter(sample)
pprint(letter_frequency)

In [ ]:
# Get normed frequency

total_letters = sum(letter_frequency.values())
norm_letter_frequency = {k: round(v/total_letters, 5) for k, v in letter_frequency.items()}
sorted_norm_letter_frequency = dict(sorted(norm_letter_frequency.items(), key=lambda item: item[1], reverse=True))
sorted_norm_letter_frequency

In [ ]:
# Plotting

plt.figure(figsize=(10, 8))  # Set the figure size to be similar to the one in the image

plt.bar(sorted_norm_letter_frequency.keys(), sorted_norm_letter_frequency.values(), color=(0.9, 0.9, 0.9), 
    edgecolor='black', linewidth=1, width=1.0)
plt.xlabel('Letters')

plt.ylabel('Frequency')

plt.title('Letter Frequency in Moby-Dick')

plt.tight_layout()  # Adjust the layout to make sure everything fits without overlapping

plt.show()

In [ ]:
# Sample text, encrypted

text1 = """Sy l nlx sr pyyacao l ylwj eiswi upar lulsxrj isr sxrjsxwjr, ia esmm
rwctjsxsza sj wmpramh, lxo txmarr jia aqsoaxwa sr pqaceiamnsxu, ia esmm caytra
jp famsaqa sj. Sy, px jia pjiac ilxo, ia sr pyyacao rpnajisxu eiswi lyypcor
l calrpx ypc lwjsxu sx lwwpcolxwa jp isr sxrjsxwjr, ia esmm lwwabj sj aqax
px jia rmsuijarj aqsoaxwa. Jia pcsusx py nhjir sr agbmlsxao sx jisr elh.
-Facjclxo Ctrramm"""

In [ ]:
# Sample text, unencrypted

text2 = """Alan Mathison Turing was a British mathematician, logician, cryptanalyst, and computer
scientist. He was highly influential in the development of computer science, providing a
formalisation of the concepts of "algorithm" and "computation" with the Turing machine. Turing
is widely considered to be the father of computer science and artificial intelligence. During
World War II, Turing worked for the Government Code and Cypher School (GCCS) at Bletchley Park,
Britain's codebreaking centre. For a time he was head of Hut 8, the section responsible for
German naval cryptanalysis. He devised a number of techniques for breaking German ciphers,
including the method of the bombe, an electromechanical machine that could find settings
for the Enigma machine. After the war he worked at the National Physical Laboratory, where
he created one of the first designs for a stored-program computer, the ACE. In 1948 Turing
joined Max Newman's Computing Laboratory at Manchester University, where he assisted in the
development of the Manchester computers and became interested in mathematical biology. He wrote
a paper on the chemical basis of morphogenesis, and predicted oscillating chemical reactions
such as the Belousov-Zhabotinsky reaction, which were first observed in the 1960s. Turing's
homosexuality resulted in a criminal prosecution in 1952, when homosexual acts were still
illegal in the United Kingdom. He accepted treatment with female hormones (chemical castration)
as an alternative to prison. Turing died in 1954, just over two weeks before his 42nd birthday,
from cyanide poisoning. An inquest determined that his death was suicide; his mother and some
others believed his death was accidental. On 10 September 2009, following an Internet campaign,
British Prime Minister Gordon Brown made an official public apology on behalf of the British
government for "the appalling way he was treated." As of May 2012 a private member's bill was
before the House of Lords which would grant Turing a statutory pardon if enacted."""

In [ ]:
# Show letter distribution, 1

Counter(preprocess(text1.lower()))

In [ ]:
# Show letter distribution, 2

Counter(preprocess(text2.lower()))

In [ ]:
# Frequency Finder
# https://www.nostarch.com/crackingcodes (BSD Licensed)

ETAOIN = 'ETAOINSHRDLCUMWFGYPBVKJXQZ'
LETTERS = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [ ]:
# Helper function; basically, a counter object

def getLetterCount(message):
    # Returns a dictionary with keys of single letters and values of the
    # count of how many times they appear in the message parameter:
    letterCount = {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0, 'G': 0, 'H': 0, 'I': 0, 'J': 0, 'K': 0, 'L': 0, 'M': 0, 'N': 0, 'O': 0, 'P': 0, 'Q': 0, 'R': 0, 'S': 0, 'T': 0, 'U': 0, 'V': 0, 'W': 0, 'X': 0, 'Y': 0, 'Z': 0}

    for letter in message.upper():
        if letter in LETTERS:
            letterCount[letter] += 1

    return letterCount

In [ ]:
# Show letter count, alphabetical order

getLetterCount(text2)

In [ ]:
# Get frequency order

def getFrequencyOrder(message):
    # Returns a string of the alphabet letters arranged in order of most
    # frequently occurring in the message parameter.

    # First, get a dictionary of each letter and its frequency count:
    letterToFreq = getLetterCount(message)
    
    return "".join([k for k, v in sorted(list(letterToFreq.items()), key=lambda x: x[1], reverse=True)]) 

getFrequencyOrder(text2)


In [ ]:
# Create "frequency match" measurement

def englishFreqMatchScore(message):
    # Return the number of matches that the string in the message
    # parameter has when its letter frequency is compared to English
    # letter frequency. A "match" is how many of its six most frequent
    # and six least frequent letters is among the six most frequent and
    # six least frequent letters for English.
    freqOrder = getFrequencyOrder(message)

    matchScore = 0
    # Find how many matches for the six most common letters there are:
    for commonLetter in ETAOIN[:6]:
        if commonLetter in freqOrder[:6]:
            matchScore += 1
    # Find how many matches for the six least common letters there are:
    for uncommonLetter in ETAOIN[-6:]:
        if uncommonLetter in freqOrder[-6:]:
            matchScore += 1

    return matchScore

In [ ]:
# Show measurement example, 1

englishFreqMatchScore(text1)

In [ ]:
# Show measurement example, 1

englishFreqMatchScore(text2)

## Caesar cipher

In [ ]:
# Encrypting a Caesar cipher

LETTERS = "abcdefghijklmnopqrstuvwxyz"

def caesar_encrypt(plaintext, key):
    ciphertext = ""
    for letter in plaintext:
        if letter.lower() in LETTERS:
            if letter.isupper():
                ciphertext += LETTERS[(LETTERS.index(letter.lower()) + key) % len(LETTERS)].upper()
            else:
                ciphertext += LETTERS[(LETTERS.index(letter.lower()) + key) % len(LETTERS)]
        else:
            ciphertext += letter
    return ciphertext

In [ ]:
# Example, 1

caesar_encrypt("hello world", 1)

In [ ]:
# Example, 2

caesar_encrypt("IBM", -1)

In [ ]:
# Caesar Cipher Hacker
# https://www.nostarch.com/crackingcodes (BSD Licensed)

message = "guv6Jv6Jz!J6rp5r7Jzr66ntrM"
SYMBOLS = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz1234567890 !?."

messages = [
    "qeFIP?eGSeECNNS,",
    "5coOMXXcoPSZIWoQI,",
    "avnl1olyD4l'ylDohww6DhzDjhuDil,",
    "z.GM?.cEQc. 70c.7KcKMKHA9AGFK,",
    "?MFYp2pPJJUpZSIJWpRdpMFY,",
    "ZqH8sl5HtqHTH4s3lyvH5zH5spH4t pHzqHlH3l5K",
    "Zfbi,!tif!xpvme!qspcbcmz!fbu!nfA",
]

# Loop through every possible key:

for message in messages[:1]:
    for key in list(range(len(SYMBOLS))):
        # It is important to set translated to the blank string so that the
        # previous iteration's value for translated is cleared.
        translated = ""

        # The rest of the program is almost the same as the original program:

        # Loop through each symbol in `message`:
        for symbol in message:
            if symbol in SYMBOLS:
                symbolIndex = SYMBOLS.find(symbol)
                translatedIndex = symbolIndex - key

                # Handle the wrap-around:
                if translatedIndex < 0:
                    translatedIndex = translatedIndex + len(SYMBOLS)

                # Append the decrypted symbol:
                translated = translated + SYMBOLS[translatedIndex]

            else:
                # Append the symbol without encrypting/decrypting:
                translated = translated + symbol

        # Display every possible decryption:
        print("Key #%s: %s" % (key, translated))


In [ ]:
# Hack cipher, with language detection

for message in messages:
    print(message)
    for key in list(range(len(SYMBOLS))):
        # It is important to set translated to the blank string so that the
        # previous iteration's value for translated is cleared.
        translated = ""

        # The rest of the program is almost the same as the original program:

        # Loop through each symbol in `message`:
        for symbol in message:
            if symbol in SYMBOLS:
                symbolIndex = SYMBOLS.find(symbol)
                translatedIndex = symbolIndex - key

                # Handle the wrap-around:
                if translatedIndex < 0:
                    translatedIndex = translatedIndex + len(SYMBOLS)

                # Append the decrypted symbol:
                translated = translated + SYMBOLS[translatedIndex]

            else:
                # Append the symbol without encrypting/decrypting:
                translated = translated + symbol

        # Display every possible decryption:
        if isEnglish(translated, 10):
            print("Key #%s: %s" % (key, translated))
    print()